In [1]:
from mindquantum.utils import random_circuit
from mindquantum.core.circuit import Circuit
from huawei_graph_tool import ConvertCircuit,my_random_circuit

import random,os,torch
import networkx as nx
from mindquantum.io.qasm import HiQASM
from torch_geometric.transforms import RandomLinkSplit
# from Pytorch_VGAE import CustomGraphDataset,VGAEModel
import pandas as pd
import torch.nn.functional as F
from torch_geometric.utils import from_scipy_sparse_matrix, dense_to_sparse
from Pytorch_VGAE import CustomGraphDataset,VGAEModel
from torch_geometric.loader import DataLoader
import numpy as np

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR


In [2]:
A_list = []
X_list = []
for i in range(5000):
    A_list.append(pd.read_csv(f'./VGAE_dataset/raw/A/A_{i}.csv', header=None).values)
    X_list.append(pd.read_csv(f'./VGAE_dataset/raw/X/X_{i}.csv', header=None).values)


In [3]:
from torch_geometric.utils import train_test_split_edges
from torch_geometric.data import Data
import scipy.sparse as sp

# 将邻接矩阵转换为 PyTorch Geometric 支持的 edge_index 格式
def adj_to_edge_index(adj):
    if isinstance(adj, np.ndarray):
        adj = sp.csr_matrix(adj)
    adj_coo = adj.tocoo()
    row = torch.tensor(adj_coo.row, dtype=torch.long)
    col = torch.tensor(adj_coo.col, dtype=torch.long)
    edge_index = torch.stack([row, col], dim=0)
    return edge_index

# 准备 PyG 数据集并分割训练/测试边
def prepare_data(X, A):
    data_list = []
    for i in range(len(X)):
        x = torch.tensor(X[i], dtype=torch.float)
        edge_index = adj_to_edge_index(A[i])
        data = Data(x=x, edge_index=edge_index)
        #data = train_test_split_edges(data)
        data_list.append(data)
    return data_list

In [26]:
data_list = prepare_data(X_list, A_list)

In [ ]:
batch_size = 32  # 你想要的批量大小
loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)
for batch in loader:
    
    print(batch)